# Perform Topic Extraction and Multi Label Classification

In [2]:
import pandas as pd 
from openai import OpenAI
import keyring
from typing import List, Union, Dict

import re
import time

import datetime
date = datetime.datetime.now().strftime("%Y-%m-%d")

from Classification_Helpers import (shuffle_extract, shuffle_extract_w_defs,
                                    wide_encode_output, getNumTokens, 
                                    get_running_cost, multiclassifyVerbatim, multiclassifyVerbatimwDefs
                                )

### Setting up OpenAI API Key

For each new project, make sure there is a new Project set up in the OpenAI NAXION organization.

  - Create a new API key for yourself in the project.
  - Store the API in Windows Credentials with the following naming scheme
    - Address: OPENAI_KEY_{JOB #}
    - Username: {JOB #}
    - Password: {API KEY}

In [4]:
# Establishing api connection to OpenAI
key = "DEVELOPMENT"
client = OpenAI(api_key=keyring.get_password(f"OPENAI_KEY_{key}", key))

# Preprocessing Data

Here we need to load in the data and get it into the proper format for the extraction and classification. This could be a different process for every new project since data is always different.

In [9]:
input_df = pd.read_csv('Input_Data/Verbatims.csv')

# Rename ID to NAID
input_df.rename(columns={'ID': 'NAID'}, inplace=True)

In [10]:
cols = input_df.drop(["NAID"], axis = 1).columns.tolist()
cols

['Verbatim']

In [ ]:
# Set extra context for the question
q_context = {
    "Q5a": "Respondents were asked to identify the company/provider and Large Language Model (LLM) they find the most relevant and to provide a brief explanation for their choice. These are the explanations provided by the respondents.",
    "Q5b": "Respondents were asked to identify the company/provider and Large Language Model (LLM) they find the least relevant and to provide a brief explanation for their choice. These are the explanations provided by the respondents."
}

In [11]:
for col in cols:
    vals = input_df[col].dropna()
    print(f"{col}: {len(vals)}")
    num_tokens = getNumTokens("|".join(vals))
    print(f"Tokens: {num_tokens}")

Verbatim: 162
Tokens: 3300


# Topic Extraction

In [ ]:
topics_dict = {}

In [ ]:
start_time = time.time()
for col in cols[1:]:
    q_time = time.time()
    normal_context = q_context[col]
    print(f'Context: {normal_context}')
    
    topics_1 = shuffle_extract(client, input_df, col, 40, normal_context)
    topics_2 = shuffle_extract(client, input_df, col, 40, normal_context)
    topics_3 = shuffle_extract(client, input_df, col, 40, normal_context)
    
    topics_dict[col] = {
        'Topics_1' : topics_1,
        'Topics_2' : topics_2,
        'Topics_3' : topics_3
    }
    
    print(f"Time for {col}: {round((time.time() - q_time) / 60,2)} minutes\n")
print("-" * 50)
print(f"Total time: {round((time.time() - start_time) / 60,2)} minutes")

## Saving to Excel

In [ ]:
# Write Staff results to an excel file in different sheets
with pd.ExcelWriter('Output/Topics/Topics_Extracted.xlsx') as writer:
    for col, dict in topics_dict.items():
        df = pd.DataFrame(dict)
        df.to_excel(writer, sheet_name=col, index=False)
        worksheet = writer.sheets[col]
        # set all columns to have width of 50
        worksheet.set_column('A:C', 50)
    writer.save()

# Multilabel Classification

In [ ]:
def write_to_excel(path, dfs, topics_list):

    # Write to excel with each sheet being a different format
    with pd.ExcelWriter(path) as writer:
        workbook = writer.book

        # Create text wrap format
        text_wrap = workbook.add_format({'text_wrap': True})
        center_wrap = workbook.add_format({'text_wrap': True, 'valign': 'vcenter', 'align': 'center'})
        center_wrap_blue = workbook.add_format({'text_wrap': True, 'valign': 'vcenter', 'align': 'center', 'bg_color': '#009999'})
        
        for df_name, df in dfs.items():
            print(df_name)
            question = df_name.split('_')[0]
            
            topics = topics_list[question]

            wide_output_binary = wide_encode_output(df, id_cols = ['NAID', question], bin_rank = 'binary') 
            col_order = ['NAID', question] + [x for x in topics if x in wide_output_binary.columns] + ['Other']  
            wide_output_binary[col_order].to_excel(writer, sheet_name=df_name, index=False)
            worksheet = writer.sheets[df_name]
            worksheet.set_column('A:A', 15, center_wrap)
            worksheet.set_column('B:B', 50, text_wrap)
            n_rows, n_cols = wide_output_binary.shape
            worksheet.set_column(2, n_cols-1, 35, center_wrap)
            # Add conditional formatting to binary sheet if the value is 1
            worksheet.conditional_format(2, 2, n_rows-1, n_cols-1, {'type':     'cell',
                                                'criteria': '=',
                                                'value':    1,
                                                'format':   center_wrap_blue})
            worksheet.freeze_panes(1, 2)  # Freeze first row and first 2 columns.
    

### Final Set of Topics

This code chunk is useful if you have copilot. The "raw q06" is what the PM provides to me, and the list_topics_106 is the format I want it in. So I keep that as an example and then I can paste the PM input into raw_topics_q39 or wahtever, and copilot will automatically generate the list topics in the output I desire. Then I put it in the list_topics dictionary

In [ ]:
raw_topics_q06 = """Remote/Hybrid Work Model: Hybrid work model, Lack of face-to-face time, Virtual connection, In-person connections
DE&I: Age diversity, Diversity and inclusion, Diversity in leadership, Inclusivity initiatives, Cultural differences, Bias awareness
Employee Training/Development: Mentorship programs, Learning circles, Career development, Professional growth, Leadership development, Professional relationships, Training opportunities, Personal growth opportunities
Culture/Values: Company culture, Feedback culture, Family-oriented culture, Cultural fit, Alignment with values, Company values, Traditional family emphasis, Corporate purpose, Open door culture, Collaborative work culture, Cathy family influence, Sense of belonging
Employee Well-Being/Care: Employee well-being, Work-life balance, Employee empowerment, Employee engagement, Recognition, Recognition practices
Peer practices: Peer relationships, Peer support, Peer interaction, Team interaction
Events/Activities: Team retreats, Social activities, In-person events, Cultural cornerstone events
Communication: Supervisory communication, Departmental communication, Direct communication
Collaboration: Team collaboration, Cross-functional projects, Departmental connection

"""

list_topics_q06 = ["Hybrid work model", "Lack of face-to-face time", "Virtual connection", 'In-person connection',
               "Age diversity", "Diversity and inclusion", "Diversity in leadership", "Inclusivity initiatives", "Cultural differences", "Bias awareness",
               "Mentorship programs", "Learning circles", "Career development", "Professional growth", "Leadership development", "Professional relationships", "Training opportunities", "Personal growth opportunities",
               "Company culture", "Feedback culture", "Family-oriented culture", "Cultural fit", "Alignment with values", "Company values", "Traditional family emphasis", "Corporate purpose", "Open door culture", "Collaborative work culture", "Cathy family influence", "Sense of belonging",
               "Employee well-being", "Work-life balance", "Employee empowerment", "Employee engagement", "Recognition", "Recognition practices",
               "Peer relationships", "Peer support", "Peer interaction", "Team interaction",
               "Team retreats", "Social activities", "In-person events", "Cultural cornerstone events",
               "Supervisory communication", "Departmental communication", "Direct communication",
               "Team collaboration", "Cross-functional projects", "Departmental connection"]

raw_topics_q39 = """
Workload: Work-Life Balance
Hybrid/Remote Model: Flexible Future Model, Hybrid Work Challenges, Home Office Setup Support, Traffic and Commute, Remote Work Stipends 
In-Person/On-site Collaboration: In-Person Collaboration, Team Days Onsite, In-Person Meetings
Technology: Remote Work Tools, Technology Upgrades, Meeting Technology Issues, Meeting Room Tech Reliability, Office Technology Consistency, Workplace Technology Consistency
Office Design/Management: Private Workspaces, Office Space Availability, Personalized Work Environment, Office Design, Office Amenities, Meeting Room Availability, 
Meeting Frequency: Meeting Effectiveness, 
Productivity: Productivity Tools, Staff Training on Productivity, Heads Down Work,
Collaboration: Team Connection Days, Teamwork and Collaboration
Culture/Values: Cultural Connection,
Training/Development: Professional Development
Management: Leadership Visibility, Staff Accountability, Team Building, Team Coordination, Team Connection, Defined Goals and Expectations, Communication: Cross-Functional Communication, Clear Communication Channels, Communication Tools, Communication Channels
Engagement: Staff Engagement, 
Cultural Events: Cornerstone Events, Cultural Events
Benefits: On-Site Childcare Options
*Safety: Workplace Safety
"""

list_topics_q39 = ["Work-Life Balance", 
                   "Flexible Future Model", "Hybrid Work Challenges", "Home Office Setup Support", "Traffic and Commute", "Remote Work Stipends",
                   "In-Person Collaboration", "Team Days Onsite", "In-Person Meetings",
                   "Remote Work Tools", "Technology Upgrades", "Meeting Technology Issues", "Meeting Room Tech Reliability", "Office Technology Consistency", "Workplace Technology Consistency",
                   "Private Workspaces", "Office Space Availability", "Personalized Work Environment", "Office Design", "Office Amenities", "Meeting Room Availability",
                   "Meeting Effectiveness",
                   "Productivity Tools", "Staff Training on Productivity", "Heads Down Work",
                   "Team Connection Days", "Teamwork and Collaboration",
                   "Cultural Connection",
                   "Professional Development",
                   "Leadership Visibility", "Staff Accountability", "Team Building", "Team Coordination", "Team Connection", "Defined Goals and Expectations", "Cross-Functional Communication", "Clear Communication Channels", "Communication Tools", "Communication Channels",
                   "Staff Engagement",
                   "Cornerstone Events", "Cultural Events",
                   "On-Site Childcare Options",
                   "Workplace Safety"]


In [ ]:
topics_list ={
    
    'Q06' : ["Hybrid work model", "Lack of face-to-face time", "Virtual connection", 'In-person connection',
               "Age diversity", "Diversity and inclusion", "Diversity in leadership", "Inclusivity initiatives", "Cultural differences", "Bias awareness",
               "Mentorship programs", "Learning circles", "Career development", "Professional growth", "Leadership development", "Professional relationships", "Training opportunities", "Personal growth opportunities",
               "Company culture", "Feedback culture", "Family-oriented culture", "Cultural fit", "Alignment with values", "Company values", "Traditional family emphasis", "Corporate purpose", "Open door culture", "Collaborative work culture", "Cathy family influence", "Sense of belonging",
               "Employee well-being", "Work-life balance", "Employee empowerment", "Employee engagement", "Recognition", "Recognition practices",
               "Peer relationships", "Peer support", "Peer interaction", "Team interaction",
               "Team retreats", "Social activities", "In-person events", "Cultural cornerstone events",
               "Supervisory communication", "Departmental communication", "Direct communication",
               "Team collaboration", "Cross-functional projects", "Departmental connection"],
    'Q39' : ["Cathy Family Leadership", "Biblical Principles", "Culture of Care", "Generosity and Giving", "Corporate Purpose", "Prayer and Devotion", "Positive Workplace Culture", "Purpose-Driven Work", "Culture of Hospitality", "Volunteerism and Philanthropy", "Brand Reputation", "Caring Company Reputation", "Positive Work Environment",
                "Ethical Business Practices", "Sustainability Efforts",
                "Cultural Ambassadorship", "Cultural Cornerstone Events",
                "DEI Initiatives", "Inclusive Workforce",
                "Employee Well-being", "Employee Benefits", "Employee Engagement", "Recognition and Rewards", "Work-Life Balance", "Team Member Recognition", "Team Member Support", "Health and Wellness Programs", "Operator Support",
                "Team Member Development", "Professional Development", "Professional Growth", "Leadership Development", "Staff Development Opportunities",
                "Community Service",
                "Remote Work Support", "Flexible Work Environment",
                "Customer Care Initiatives", "Customer Service Excellence", "Customer Feedback", "Growth and Adaptation", "Continuous Improvement", "Innovation and Growth", "Cultural Adaptation", "Cultural Drift Concerns",
                "Employee Feedback Mechanisms", "Transparent Communication",
                "Purpose-Driven Hiring",
                "Employee Onboarding Experience",
                "Team Collaboration", "Collaborative Culture",
                "Strategic Priorities", "Strategic Decision Making", "Scholarship Programs",
                "Servant Leadership", "Caring Leadership", "Stewardship of Resources"],
    'Q40' : ["DE&I Initiatives",
              "Remote Work, Flexible Futures Model",
              "Staff Onboarding", "Essentials Program",
              "Cathy Family Involvement", "Truett's Legacy",
              "Core Values", "Corporate Purpose", "Cultural Reinforcement", "Cultural Consistency", "Biblical Principles", "Purpose-Driven Work", "Cultural Storytelling", "Cultural Cornerstones", "Staff Accountability",
              "Cultural Celebrations", "Cultural Events", "Cultural Cornerstone Events", "Cultural Ambassadors", "Cultural Adaptation", "Founder's Day", "Staff Summit", "NEXT",
              "Cultural Fit", "Hiring Practices", "Cultural Training", "Talent Selection",
              "Team Member Support", "Staff Care", "Team Member Care", "Employee Well-being", "Work-Life Balance", "Employee Empowerment", "Employee Recognition", "Team Retreats", "Employee Benefits",
              "Community Service", "Community Engagement", "Staff-Community Engagement",
              "Employee Feedback", "Employee Development", "Mentorship Programs", "Leadership Development", "Team Development", "Accountability & Feedback", "Recognition Programs", "Personal Connection Opportunities", "Professional Development",
              "Transparent Communication", "Transparency in Decision Making",
              "Customer Focus", "Customer Experience",
              "Operator Engagement", "Staff-Operator Connection"]
}

### Perform the Classification

In [ ]:
classified_dfs = {}

In [ ]:
start_time = time.time()
start_cost = get_running_cost()

for question in cols:
    q_time = time.time()
    q_cost = get_running_cost()
    print(f'\n\nQuestion {question}')
    
    
    df = input_df.loc[:,['NAID', question]].dropna().reset_index(drop=True)
    
    print(f'Number of Verbatims: {df.shape[0]}')
    print('---------------------------------')
    
    context = q_context[question]
    topics = sorted(cols[question])
    
    raw_labels: pd.DataFrame = df.apply(lambda row: multiclassifyVerbatim(client, row[question], topics= topics, 
                                                                        context=context, 
                                                                        id=row['NAID'], i=row.name, max_labels=3, n_resp = 1), axis=1)
    
    raw_labels_df = pd.concat(raw_labels.to_list(), ignore_index=True).rename(columns={'ID': 'NAID'})

    classified_dfs[question] = pd.merge(df, raw_labels_df)
    classified_dfs[question].to_pickle(f'Output/Pickles/Topics_Classified_{question}_{date}.pkl')
    
    print('---------------------------------')
    print(f'Question {question} took {time.time() - q_time:.2f} seconds')
    print(f'Cost of Question {question}: ${get_running_cost() - q_cost:.2f}')


print(f'\n\n\nTotal Time: {time.time() - start_time:.2f} seconds')
print(f'Total Cost: ${get_running_cost() - start_cost:.2f}')

In [ ]:
# ldate = '2024-02-15'
# for question in cols + ['Q02a_Agree', 'Q02a_Disagree', 'Q04a_Agree', 'Q04a_Disagree', 'Q07a_Agree', 'Q07a_Disagree', 'Q09a_Agree', 'Q09a_Disagree', 'Q15_Paycheck', 'Q15_Passion', 'Q15_Neutral']:
#     staff_classified_dfs[question] = pd.read_pickle(f'Output/Pickles/Staff_Topics_Classified_{question}_{ldate}.pkl')

In [ ]:
write_to_excel(f'Output/Labels/Topics_Classified_{date}.xlsx', classified_dfs, topics_list)

## Perform Classification with Defintions

In [1]:
topics_defs_dict = {
    'Q5a' :  {
        'Accurate, Reliable, and Unbiased Performance' :  'The accuracy, reliability, and fairness of the models in performing tasks without bias.',
        'Advanced AI/Machine Learning Features' :  'The provider and or model is a sophisticated AI and has advanced machine learning capabilities',
        'Advanced Natural Language Processing' :  'The model has advanced NLP capabilities, including summarization, reasoning, content generation, explanation, complex understanding, etc',
        'Advanced Risk Management' :  "The model and or provider manages risks effectively",
        'Automated Content Generation' :  "The model is good at automatically generating content, such as text or code",
        'Cloud-Based Solutions' :  'Benefits associated with using the Cloud',
        'Extensive Documentation' :  'There is detailed documentation for the models.',
        'Cost Reduction' :  'The model reduces costs',
        'Cost-Effective Solutions and Deployment' :  'The model is affordable and provides economic efficiency in usage and or deployment',
        'Customizable and Flexible' :  "The model is flexibile and customizable",
        'Easy Integration' :  "The model can be easily integrated with the respondent's existing systems and software",
        'Efficient Data Analysis and Management' :  "The model has capabilities in handling and analyzing datasets efficiently",
        'Enhanced Customer Support' :  'The model enhances customer support services, including through automation',
        'Enhanced Data Security and Compliance' :  'The model and or provider has robust security and compliance standards',
        'Enhanced Marketing and Communication' :  'The models improve marketing efforts and communication strategies',
        'Existing Relationship with Provider' :  'The respondent has previously or is currently using products by the provider',
        'Familiarity with the Provider/Model' :  'The respondent is familiar with the provider and or models, or has used them before.',
        'Fit and Relevance to Needs' :  'The model and or provider fits the needs of the respondent or is relevant to their business. The model has relevant features and capabilities.',
        'High-Performance Computing' :  "The model is capable to perform high-level computing tasks, managing complex calculations and data processing efficiently.",
        'Improve Business Intelligence and Analytics' :  ' The enhancement of business intelligence and analytics capabilities through the use of advanced models.',
        'Improved Developer and Programming Efficiency' :  "The model contributes to enhancing the efficiency of developers and programmers through automation and advanced tools.",
        'Improved Operational Efficiency' :  'The model contributes to streamlining operations and improving overall efficiency.',
        'Industry-Specific Applications' :  "The model has applicability and customization to the specific industries of the respondent",
        'Innovative' :  'The model and or provider is innovative',
        'Multi-Language Support' :  "The model can support multiple languages",
        'Open-Source Availability' :  'The model is in an open-source format and is available for public use',
        'Provider Customer Support' :  'The provider has good customer support services',
        'Real-Time Processing and Rapid Updates' :  'The model is very fast and can process real-time data',
        'Scalability' :  "Model has ability to scale according to the needs of the business",
        'Supports Multimedia Data' :  "Ability to handle and process multimedia data, including text, images, and videos.",
        'Trusted Brand Reputation and Market Leader' :  'The provider has a strong market presence,  trusted reputation, or is a market leader.',
        'User-Friendly Interface and Management' :  'Ease of use and intuitive interfaces of the models or platforms',
        'Wide Applicability' :  'Broad applicability of the models across various tasks and industries.',
        'Widespread Adoption and Popularity' : 'The model is widely adopted and recognized for it\' popularity in the market'
        },
    
    'Q5b': 
        {
        'Bad Past Experience with Provider' : 'There has been a bad past experience with the AI provider.',
        'Current Provider Meets Needs' : 'The current AI provider meets the needs, so switching is unnecessary.',
        'Difficult to Integrate' : 'There are difficulties in integrating the AI solution with existing systems and workflows.',
        'Difficult to Manage' : 'The AI model is difficult to manage effectively.',
        'Difficult to Use' : 'The AI model is not user-friendly or easy to use, including difficulty to train.',
        'Handles Large Data Poorly' : 'The AI model handles large data sets poorly.',
        'Hardware Requirements' : 'The AI model requires significant computational resources, which is seen as a limitation.',
        'High Cost' : 'There is concern about the high cost of using the AI model or service.',
        'Inaccurate Performance' : 'There is concern about the performance accuracy of the AI model.',
        'Inferior Model' : 'The AI model is considered inferior to other available models, including performance or being too small',
        'Irrelevant Features' : 'The AI model has features that are not relevant or useful.',
        'Irrelevant for Business' : 'The AI model is not relevant or suitable for the respondent\'s business needs.',
        'Lack of Customization' : 'It is challenging to customize the AI model to meet specific needs.',
        'Lack of Familiarity/Relationship with Provider' : 'The respondent is not familiar with the AI model or provider, or does not have a relationship with them',
        'Lack of Innovation' : 'There is a lack of innovation or advancement in the AI model.',
        'Lacks Advanced Capabilities' : 'There is a lack of advanced capabilities in the AI model such as summarization, content generation, explanation, complex understanding.',
        'Lacks Strategic Direction' : 'The AI model lacks strategic direction or clear guidance.',
        'No Multi-Language Support' : 'The AI model offers limited support for multiple languages, reducing its utility.',
        'No Multimedia Support' : 'The AI model lacks support for multimedia content like images, and just handles text data.',
        'Not Flexible' : 'The AI model is less versatile or flexible for various applications.',
        'Not Industry Specific' : 'The AI model is designed for a specific industry, limiting its broader applicability.',
        'Not Open Source' : 'The AI model is not open source, limiting its accessibility.',
        'Not Scalable' : 'The AI model has scalability issues.',
        'Not Widely Recognized': 'The AI model is not widely recognized or used in mainstream AI research.',
        'Nothing Wrong' : 'There is no specific issue or concern with the AI model or provider, including having never used it before or not currently using it.',
        'Poor Customer Support' : 'The customer support from the AI provider is inadequate.',
        'Provider Has Bad Reputation' : 'There are concerns about the reputation of the AI provider.',
        'Provider Not a Leader in AI' : 'The provider is not widely recognized or used in mainstream AI research.',
        'Security and Ethics Concerns' : 'There are ethical and safety concerns related to the use of the AI model.',
        'Slow Performance': 'There is concern about the slow performance or responsiveness of the AI model.',
        }
        }

In [ ]:
c_w_defs = {}

In [ ]:
start_time = time.time()
start_cost = get_running_cost()

for question in cols[:1]:
    q_time = time.time()
    q_cost = get_running_cost()
    print(f'\n\nQuestion {question}')
    
    
    df = input_df.loc[:,['NAID', question]].dropna().reset_index(drop=True)
    
    print(f'Number of Verbatims: {df.shape[0]}')
    print('---------------------------------')
    
    context = q_context[question] 
    # if question == 'Q5a':
        # context += + "Note that anything referencing a capability of an LLM model such as 'reasoning ability', 'summarizing', 'understanding' should include the classification as 'Advanced LLM Capabilities'."
    topics = topics_defs_dict[question]
    print(topics)
    
    raw_labels: pd.DataFrame = df.apply(lambda row: multiclassifyVerbatimwDefs(client, row[question], topics= topics, 
                                                                        context=context, 
                                                                        id=row['NAID'], model = "gpt-4-0125-preview", i=row.name, max_labels=3, n_resp = 1), axis=1)
    
    raw_labels_df = pd.concat(raw_labels.to_list(), ignore_index=True).rename(columns={'ID': 'NAID'})

    c_w_defs[question] = pd.merge(df, raw_labels_df)
    c_w_defs[question].to_pickle(f'Output/Pickles/Topics_Classified_{question}_{date}.pkl')
    
    print('---------------------------------')
    print(f'Question {question} took {time.time() - q_time:.2f} seconds')
    print(f'Cost of Question {question}: ${get_running_cost() - q_cost:.2f}')


print(f'\n\n\nTotal Time: {time.time() - start_time:.2f} seconds')
print(f'Total Cost: ${get_running_cost() - start_cost:.2f}')

In [ ]:
write_to_excel(f'Output/Labels/Topics_Classified_w_Defs_{date}.xlsx', c_w_defs, {key: list(topics_defs_dict[key].keys()) for key in topics_defs_dict.keys()})